# FPP multi objective optimization

### Import packages

In [ ]:
#@time using WarmupFUSE
@time using FUSE
using Plots; gr();
FUSE.logging(Logging.Info; actors=Logging.Error);

### Setup distributed computing environment

See more details here: https://fuse.help/parallel.html

In [ ]:
FUSE.parallel_environment("saga", 120)
using Distributed
#@time @everywhere import WarmupFUSE
@time @everywhere import FUSE

### Get `ini` and `act` for FPP case and custmize as needed

In [ ]:
ini, act = FUSE.case_parameters(:FPP; version=:v1_demount, init_from=:scalars, STEP=true)
act.ActorPFcoilsOpt.optimization_scheme = :none; # don't spend time optimizing the PFs

### As a good practice, test the actor/workflow that you want to optimize first

In [ ]:
@time dd = FUSE.init(ini, act)
@time FUSE.ActorWholeFacility(dd, act);

In [ ]:
#IMAS.freeze(dd.balance_of_plant)
plot(dd.core_profiles)

### Define optimization variables and ranges

In [ ]:
# nominal value and ranges
ini_opt = deepcopy(ini)
ini_opt.ec_launchers.power_launched = ini.ec_launchers.power_launched[1] ↔ [10e6, 200e6];
ini_opt.core_profiles.zeff = ini.core_profiles.zeff ↔ [1.1, 2.5]
#ini_opt.equilibrium.δ = ini.equilibrium.δ ↔ [-0.7,0.7]
#ini_opt.equilibrium.ζ = ini.equilibrium.ζ ↔ [0,0.2]
ini_opt.equilibrium.κ = missing # k set to be 95% of maximum controllable value
ini_opt.equilibrium.B0 = ini.equilibrium.B0 ↔ [1.0, 20.]
ini_opt.equilibrium.ip = ini.equilibrium.ip ↔ [1.0e6, 22e6]
ini_opt.equilibrium.R0 = ini.equilibrium.R0 ↔ [ini.equilibrium.R0, 10.0];

### Define the optimization objectives

In [ ]:
# FUSE comes with a library of objective functions
OFL = deepcopy(FUSE.ObjectivesFunctionsLibrary)

In [ ]:
OFL[:max_power_electric_net].target = 200.0
objective_functions = [OFL[:min_βn], OFL[:min_capital_cost]]

CFL = deepcopy(FUSE.ConstraintFunctionsLibrary)
CFL[:target_power_electric_net].limit = 200.0
CFL[:target_power_electric_net].tolerance = 0.01
constraint_functions = [CFL[:target_power_electric_net], CFL[:steady_state]]

# ...but one can define custom objectives and constraints too
# target_power_electric = FUSE.ObjectiveFunction(:target_power_electric_net, "MW", dd -> @ddtime(dd.balance_of_plant.power_electric_net)/1E6, 200)
#objective_functions = [target_power_electric, OFL[:min_cost], OFL[:max_flattop]]

display(objective_functions)
display(constraint_functions)

### Setup and run optimization

In [ ]:
# option to resume an optimization where it was left off
if false
    continue_results = results
else
    continue_results = missing
end

# define optimization parameters
# For real optimization studies the population size (N) and number of iterations should be bigger
# eg. N=100, iterations=25
optimization_parameters = Dict(
    :N => max(4, Int(floor((nprocs()-1)/2))*2), # even number
    :iterations => 100,
    :continue_results => continue_results,
    :save_folder => "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta")

# run optimization
results = FUSE.workflow_multiobjective_optimization(ini_opt, act, FUSE.ActorWholeFacility, objective_functions, constraint_functions; optimization_parameters...);

## Release distributed resources

In [ ]:
for i in workers()
    rmprocs(i)
end

### Save optimization results to file

In [ ]:
# Optimization results can be re-loaded this way:
filename = "optimization_runs/optimization.bson"
@time results = FUSE.load_optimization(filename);

### How to: Define and use a custom FUSE workflow

In [ ]:
# Here `@everywhere` is needed to make all processes aware of the custom function
@everywhere function workflow_custom(ini, act)
    FUSE.init(dd, ini, act)
    FUSE.ActorEquilibriumTransport(dd, act)
    FUSE.ActorCXbuild(dd, act)
    return dd
end

# results = FUSE.workflow_multiobjective_optimization(ini, act, custom_workflow, objective_functions; optimization_parameters...);